In [1]:
from rnamodif.architectures.rodan_pretrained import RodanPretrained
from rnamodif.architectures.rodan_pretrained_MIL import RodanPretrainedMIL

model = RodanPretrainedMIL(pretrained_lr=1e-5, my_layers_lr=1e-4, warmup_steps = 10000)


TODO fix rodan path


In [2]:
from rnamodif.data_utils.dataloading2 import nanopore_datamodule
from rnamodif.data_utils.split_methods import get_kfold_splits, get_fullvalid_split, get_valid_portions

#TODO shuffle training files for workers when continuting from checppoint - so i dont go over the same files as before

dm = nanopore_datamodule(
    splits = get_kfold_splits(
        pos_exps=['s4U_33_covid'],
        neg_exps=['s4U_0_covid'], 
        total_k=5, 
        current_k=0
    )+get_fullvalid_split()(pos_files=['s4U_5_covid', 's4U_10_covid'], neg_files=[]),
    verbose=1, 
    batch_size=32, 
    valid_limit=1000, 
    workers=16,
    window=4096,
    normalization='rodan',
    
)


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

experiment_name = 's4u_0v33_big'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}", 
    save_top_k=2, 
    monitor="valid_loss", 
    save_last=True, 
    save_weights_only=True
)
logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,
    callbacks=[checkpoint_callback])
    # resume_from_checkpoint=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/m6a_all_mix_deploy/epoch=0-step=241500.ckpt")

trainer.fit(model, dm)


CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                  | Params
----------------------------------------------------------
0 | trainable_rodan | network               | 10.7 M
1 | original_rodan  | network               | 10.7 M
2 | seq_model       | Sequential            | 51    
3 | softmax         | Softmax               | 0     
4 | acc             | BinaryAccuracy        | 0     
5 | cm              | BinaryConfusionMatrix | 0     
----------------------------------------------------------
10.7 M    Trainable params
10.7 M    Non-trainable params
21.3 M    Total params
42.679    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 2 NaN values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experime

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]